In [ ]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

from random import shuffle

%matplotlib inline

In [ ]:
CARD_RANKING = {'Hn': 1, 'Hk': 2, 'Ha': 3,
                'Sn': 4, 'Sk': 5, 'St': 6, 'Sa': 7,
                'Cn': 8, 'Ck': 9, 'Ct': 10, 'Ca': 11, 
                'Dn': 12, 'Dk': 13, 'Dt': 14, 'Da': 15,
                'Dj': 16, 'Hj': 17, 'Sj': 18, 'Cj': 19,
                'Dq': 20, 'Hq': 21, 'Sq': 22, 'Cq': 23,
                'Ht': 24}

def change_key_and_values(mydict):
    """
    Changes keys and values of given dict
    Args:
        mydict: Dictionary containing key value pairs.
    Returns:
        mydict_inverse: Inverse dict.
    """
    mydict_inverse = {}
    
    for key, value in mydict.iteritems():
        mydict_inverse[value] = key
        
    return mydict_inverse

CARD_RANKING_INVERSE = change_key_and_values(CARD_RANKING)

CARD_CATEGORIES = {'hearts_fail' : ['Hn', 'Hk', 'Ha'],
                   'spades_fail' : ['Sn', 'Sk', 'St', 'Sa'],
                   'clubs_fail' : ['Cn', 'Ck', 'Ct', 'Ca'],
                   'foxes' : ['Ca']}

In [ ]:
def change_key_and_values(mydict):
    """
    Changes keys and values of given dict
    Args:
        mydict: Dictionary containing key value pairs.
    Returns:
        mydict_inverse: Inverse dict.
    """
    mydict_inverse = {}
    
    for key, value in mydict.iteritems():
        mydict_inverse[value] = key
        
    return mydict_inverse


def create_deck(nines=False, mix=True):
    """
    This function will create a full doko deck with or without nines.
    Nomenclature: 
        H: hearts, D: diamonds, S: spades, C: clubs
        (n: 9) t: 10, j: jack, q: queen, k: king, a: ace
    Args:
        nines: If True, the deck will contain nines.
        mix: If True, the deck will be shuffeled randomly.
    Returns:
        complete_deck: List of all cards in a deck.
    """
    single_deck = ['Ht', 'Hj', 'Hq', 'Hk', 'Ha',
                   'Dt', 'Dj', 'Dq', 'Dk', 'Da',
                   'St', 'Sj', 'Sq', 'Sk', 'Sa',
                   'Ct', 'Cj', 'Cq', 'Ck', 'Ca']
    
    if nines is True:
        single_deck.extend(['Hn', 'Dn', 'Sn', 'Cn'])
        
    complete_deck = 2*single_deck
    
    if mix is True:
        shuffle(complete_deck)
    else:
        complete_deck = sort_deck(complete_deck)
            
    return np.array(complete_deck)


def sort_cards(cards):
    """
    Sorts given amount of cards. This function uses 
    the global variables CARD_RANKING and CARD_RANKING_INVERSE
    Args:
        deck: Arbitrary amount of cards.
    Returns:
        sorted_deck: A sorted deck.
    """
    card_ranks = [CARD_RANKING[card] for card in cards]
    sorted_card_ranks = sorted(card_ranks)
    sorted_cards = [CARD_RANKING_INVERSE[rank] for rank in sorted_card_ranks]
    
    return np.array(sorted_cards)


def create_sample(num_decks, nines=False):
    """
    Creates a complete test sample of cards, decks and hands in pandas
    DataFrame format.
    Args: 
        num_decks: Number of decks to create
        nines: If True, decks will be created with nines
    Returns:
        data: DataFrame containing decks and player hands
    """
    decks = np.array([create_deck(nines=nines) for _ in xrange(num_decks)])
    player_hands = decks.reshape(-1, 4, len(decks[0]) / 4)

    # Sort all player hands
    for hands in player_hands:
        for i, hand in enumerate(hands):
            hands[i]  = sort_cards(hand)
            
    # Create Pandas DataFrame
    data = pd.DataFrame()
    data["decks"] = [decks[i] for i in range(num_decks)]

    for i in range(4):
        data["hand_{}".format(i+1)] = [player_hands[j, i, :] for j in range(num_decks)]
        
    return data

In [ ]:
# These functions are used for analysis of hands

def num_cards_in_cards_type(hand, card_category):
    cnt = 0
    for card in hand:
        if card in card_category: cnt += 1
    return cnt

def analyse_games(games):
    for cat_name, cat in CARD_CATEGORIES.iteritems():
        # For all hands in one game
        for j in range(4):
            games["hand_{}_num_{}".format(j + 1, cat_name)] = \
            games["hand_{}".format(j + 1)].apply(
                lambda x: num_cards_in_cards_type(x,cat))

    return games

In [ ]:
num_samples = 10000

games = create_sample(num_samples)
games = analyse_games(games)

In [ ]:
fig, ax = plt.subplots()

x_bins = np.linspace(0, 7, 8)

games.hand_1_num_hearts_fail.hist(normed=True, bins=x_bins, label='Herz fehl', color='r')
games.hand_1_num_clubs_fail.hist(normed=True, bins=x_bins, alpha=0.5, label='Kreuz fehl')
# games.hand_1_num_spades_fail.hist(normed=True, bins=x_bins, alpha=0.5, label='Pik fehl')
# games.hand_1_num_foxes.hist(normed=True, bins=x_bins, label='Fuechse', color='r')


bin_centers = 0.5 * np.diff(x_bins) + x_bins[:-1]


# Super ugly, but it works, so its fine
for bin_center, bin_val in zip(bin_centers, x_bins):
    ax.annotate(str(int(bin_val)), xy=(0 + bin_center / max(x_bins), -0.07), xycoords='axes fraction',
                horizontalalignment='center', verticalalignment='center')

ax.xaxis.set_ticks([])
ax.set_xlabel("Anzahl Karten")
ax.set_ylabel("Wahrscheinlichkeit")
ax.xaxis.set_label_coords(0.5, -0.12)
ax.legend()

fig.savefig("herz_kreuz_fehl.pdf")